In [1]:
import saspy
sas = saspy.SASsession(cfgname='sdssas', results='html')
sas

SAS Connection established. Subprocess id is 16745

No encoding value provided. Will try to determine the correct encoding.
Setting encoding to iso8859_15 based upon the SAS session encoding value of latin9.



Access Method         = STDIO
SAS Config name       = sdssas
SAS Config file       = /opt/tom/github/saspy/saspy/sascfg_personal.py
WORK Path             = /sastmp/SAS_work5BBE00004185_tom64-5/
SAS Version           = 9.04.01M6D11072018
SASPy Version         = 3.1.9
Teach me SAS          = False
Batch                 = False
Results               = html
SAS Session Encoding  = latin9
Python Encoding value = iso8859_15
SAS process Pid value = 16773


## create a dataframe with datetimes to start with

In [2]:
import pandas as pd
import datetime
rows = [[datetime.datetime(1965, 1, 1, 8, 0, 1), datetime.datetime(1965, 1, 1, 8, 0, 1), datetime.datetime(1965, 1, 1, 8, 0, 1)], 
        [datetime.datetime(1966, 1, 1, 7, 0, 2), datetime.datetime(1966, 1, 1, 7, 0, 2), datetime.datetime(1966, 1, 1, 7, 0, 2)], 
        [datetime.datetime(1967, 1, 1, 6, 0, 3), datetime.datetime(1967, 1, 1, 6, 0, 3), datetime.datetime(1967, 1, 1, 6, 0, 3)], 
        [None, None, None], 
       ]
df = pd.DataFrame.from_records(rows, columns=['dt','d','t'])
df

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01 08:00:01,1965-01-01 08:00:01
1,1966-01-01 07:00:02,1966-01-01 07:00:02,1966-01-01 07:00:02
2,1967-01-01 06:00:03,1967-01-01 06:00:03,1967-01-01 06:00:03
3,NaT,NaT,NaT


In [3]:
df.dtypes

dt    datetime64[ns]
d     datetime64[ns]
t     datetime64[ns]
dtype: object

## create SAS dataset with datetime, date, time columns 
## using datetimes={} to create the date only and time only columns
### set results to either html or text, so we see the SAS output

In [4]:
sd = sas.df2sd(df, 'sd_dt', datetimes={'d':'date', 't':'time'},results='text')
sd

Libref  = WORK
Table   = sd_dt
Dsopts  = {}
Results = text

In [5]:
sd.columnInfo()

                                                           The SAS System                       14:51 Thursday, January 16, 2020   1

                                                       The CONTENTS Procedure

                                            Alphabetic List of Variables and Attributes
 
                                            #    Variable    Type    Len    Format

                                            2    d           Num       8    E8601DA.   
                                            1    dt          Num       8    E8601DT26.6
                                            3    t           Num       8    E8601TM.   


In [6]:
sd.head()



                                                           The SAS System                       14:51 Thursday, January 16, 2020   2

                                    Obs    dt                                d            t

                                     1     1965-01-01T08:00:01.000000    1965-01-01    08:00:01
                                     2     1966-01-01T07:00:02.000000    1966-01-01    07:00:02
                                     3     1967-01-01T06:00:03.000000    1967-01-01    06:00:03
                                     4                              .             .           .


## Now, import that SAS data to pandas and see that pandas adds in the missing date or time part for cols d and t

In [7]:
df_dt = sd.to_df()
df_dt

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01,2020-01-16 08:00:01
1,1966-01-01 07:00:02,1966-01-01,2020-01-16 07:00:02
2,1967-01-01 06:00:03,1967-01-01,2020-01-16 06:00:03
3,NaT,NaT,NaT


In [8]:
df_dt.dtypes

dt    datetime64[ns]
d     datetime64[ns]
t     datetime64[ns]
dtype: object

### curious that for date, pandas only shows the date part, but the time is there, just 0 or midnight

In [9]:
df_dt['d'][0]

Timestamp('1965-01-01 00:00:00')

# CASE 1: round tripping these as numeric.
## 1) override formats when reading
## 2) send back to SAS
## 3) assign date/time formats to new data set

### lets use a different SASdata object for this case
### could have just set dsopts on sd, but I want to use seperate ones for each case
### sd.dsopts={'format':'d best32. t best32.'}

In [10]:
sd_num = sas.sasdata('sd_dt', dsopts={'format':'d best32. t best32.'})
sd_num

Libref  = WORK
Table   = sd_dt
Dsopts  = {'format': 'd best32. t best32.'}
Results = html

In [11]:
sd_num.head()

Obs,dt,d,t
1,1965-01-01T08:00:01.000000,1827,28801
2,1966-01-01T07:00:02.000000,2192,25202
3,1967-01-01T06:00:03.000000,2557,21603
4,.,.,.


In [12]:
df_num = sd_num.to_df()
df_num

,dt,d,t
0,1965-01-01 08:00:01,1827.0,28801.0
1,1966-01-01 07:00:02,2192.0,25202.0
2,1967-01-01 06:00:03,2557.0,21603.0
3,NaT,NaN,NaN


In [13]:
df_num.dtypes

dt    datetime64[ns]
d            float64
t            float64
dtype: object

## Now send back to SAS and then assign the formats and then we have what we started with

In [14]:
sd_num_ret = sas.df2sd(df_num, 'sd_num_ret')
sd_num_ret 

Libref  = WORK
Table   = sd_num_ret
Dsopts  = {}
Results = html

In [15]:
sd_num_ret.head()

Obs,dt,d,t
1,1965-01-01T08:00:01.000000,1827,28801
2,1966-01-01T07:00:02.000000,2192,25202
3,1967-01-01T06:00:03.000000,2557,21603
4,.,.,.


In [16]:
sd_num_ret.columnInfo()

### Note that I can add an option on df2sd(..., outfmts={}) for this:
### sd_num_ret = sas.df2sd(df_num, 'sd_num_ret', outfmts={'d':'date.','t':'time.'}
### then this step wouldn't be needed

In [17]:
sas.submitLOG("""
proc datasets dd=work;
 modify sd_num_ret;
  format d E8601DA. t E8601TM.;
 run;
quit;
""")



283  ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods
283! graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
284  
285  
286  proc datasets dd=work;
287   modify sd_num_ret;
288    format d E8601DA. t E8601TM.;
289   run;
NOTE: MODIFY was successful for WORK.SD_NUM_RET.DATA.
290  quit;
NOTE: PROCEDURE DATASETS used (Total process time):
      real time           0.15 seconds
      cpu time            0.11 seconds
      
291  
292  
293  ods html5 (id=saspy_internal) close;ods listing;

294  


In [18]:
sd_num_ret.head()

Obs,dt,d,t
1,1965-01-01T08:00:01.000000,1965-01-01,08:00:01
2,1966-01-01T07:00:02.000000,1966-01-01,07:00:02
3,1967-01-01T06:00:03.000000,1967-01-01,06:00:03
4,.,.,.


In [19]:
sd_num_ret.columnInfo()

# CASE 2: convert to date or time only types in Pandas (type becomes 'Object')
## 1) import the data
## 2) convert to date or time types so that what you see in pandas
## 3) convert back to datetime to send back to SAS, using datetypes={} to only get date and time

### create new df for this case

In [20]:
df_conv = sd.to_df()
df_conv

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01,2020-01-16 08:00:01
1,1966-01-01 07:00:02,1966-01-01,2020-01-16 07:00:02
2,1967-01-01 06:00:03,1967-01-01,2020-01-16 06:00:03
3,NaT,NaT,NaT


In [21]:
df_conv.dtypes

dt    datetime64[ns]
d     datetime64[ns]
t     datetime64[ns]
dtype: object

### convert to date or time only type

In [22]:
nat = pd.to_datetime('')
df_conv['d'] = df_conv['d'].apply(lambda x: x if x is nat else pd.Timestamp.date(x))
df_conv['t'] = df_conv['t'].apply(lambda x: x if x is nat else pd.Timestamp.time(x))
df_conv

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01,08:00:01
1,1966-01-01 07:00:02,1966-01-01,07:00:02
2,1967-01-01 06:00:03,1967-01-01,06:00:03
3,NaT,NaT,NaT


### can see the new types

In [23]:
df_conv.dtypes

dt    datetime64[ns]
d             object
t             object
dtype: object

In [24]:
df_conv['d'][0]

datetime.date(1965, 1, 1)

In [25]:
df_conv['t'][0]

datetime.time(8, 0, 1)

### to round trip this to SAS
### 1) convert these back to datetimes
### 2) and use datetimes={} to get only date or time

In [26]:
df_conv['d'] = pd.to_datetime(df_conv['d'].astype('str'), errors='coerce')
df_conv['t'] = pd.to_datetime(df_conv['t'].astype('str'), errors='coerce')

In [27]:
df_conv.dtypes

dt    datetime64[ns]
d     datetime64[ns]
t     datetime64[ns]
dtype: object

In [28]:
df_conv

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01,2020-01-16 08:00:01
1,1966-01-01 07:00:02,1966-01-01,2020-01-16 07:00:02
2,1967-01-01 06:00:03,1967-01-01,2020-01-16 06:00:03
3,NaT,NaT,NaT


In [29]:
sd_conv_ret = sas.df2sd(df_conv, 'df_conv', datetimes={'d':'date', 't':'time'})
sd_conv_ret

Libref  = WORK
Table   = df_conv
Dsopts  = {}
Results = html

In [30]:
sd_conv_ret.head()

Obs,dt,d,t
1,1965-01-01T08:00:01.000000,1965-01-01,08:00:01
2,1966-01-01T07:00:02.000000,1966-01-01,07:00:02
3,1967-01-01T06:00:03.000000,1967-01-01,06:00:03
4,.,.,.


In [31]:
sd_conv_ret.columnInfo()

## and to compare w/ the Numeric round trip and see they're the same

In [32]:
sd_num_ret.head()

Obs,dt,d,t
1,1965-01-01T08:00:01.000000,1965-01-01,08:00:01
2,1966-01-01T07:00:02.000000,1966-01-01,07:00:02
3,1967-01-01T06:00:03.000000,1967-01-01,06:00:03
4,.,.,.


In [33]:
sd_num_ret.columnInfo()